In [148]:
import pandas as pd
import numpy as np
import re

In [149]:
data = pd.read_csv("../data/new_laptop_data.csv", index_col=0)

In [150]:
data.head()

,Rating,MRP,Processor,RAM,OS,Storage,Brand
0,4.2,25490.0,Intel Celeron Dual Core Processor,4 GB LPDDR4 RAM,NaN,NaN,HP
1,3.8,22490.0,NaN,4 GB LPDDR4X RAM,NaN,NaN,HP
2,4.1,25685.0,Intel Celeron Dual Core Processor,8 GB DDR4 RAM,64 bit Windows 11 Operating System,256 GB SSD,Lenovo
3,4.4,51990.0,Intel Core i5 Processor,8 GB DDR4 RAM,NaN,512 GB SSD,ASUS
4,4.2,38990.0,Intel Core i3 Processor,8 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,ASUS


In [151]:
isnull = data.isnull().sum()
isnull

Rating       216
MRP            0
Processor    126
RAM            0
OS           279
Storage       87
Brand          0
dtype: int64

**Replace null values with mode**

In [152]:
def mode(col):
    return list(col.value_counts().items())[0][0]

cols = isnull[isnull != 0].index
for col in cols:
    m = mode(data[col])
    data[col].fillna(m, inplace=True)
    print(m)

data.isnull().sum()

4.2
Intel Core i5 Processor
64 bit Windows 11 Operating System
512 GB SSD


Rating       0
MRP          0
Processor    0
RAM          0
OS           0
Storage      0
Brand        0
dtype: int64

In [153]:
data.head()

,Rating,MRP,Processor,RAM,OS,Storage,Brand
0,4.2,25490.0,Intel Celeron Dual Core Processor,4 GB LPDDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,HP
1,3.8,22490.0,Intel Core i5 Processor,4 GB LPDDR4X RAM,64 bit Windows 11 Operating System,512 GB SSD,HP
2,4.1,25685.0,Intel Celeron Dual Core Processor,8 GB DDR4 RAM,64 bit Windows 11 Operating System,256 GB SSD,Lenovo
3,4.4,51990.0,Intel Core i5 Processor,8 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,ASUS
4,4.2,38990.0,Intel Core i3 Processor,8 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,ASUS


In [154]:
cols = data.dtypes[data.dtypes == "object"].index
for col in cols:
    data[col] = data[col].apply(lambda x: x.strip())

**Extract os name**

In [155]:
data.OS.value_counts()

64 bit Windows 11 Operating System    620
64 bit Windows 10 Operating System     79
32 bit Chrome Operating System         19
64 bit Chrome Operating System          1
64 bit DOS Operating System             1
Name: OS, dtype: int64

In [156]:
reg = re.compile(r"(\d{2}) bit (.*) Operating System")
reg.match("64 bit Windows 11 Operating System").groups()

data["OS_arch"] = data.OS.apply(lambda x: reg.match(x).groups()[0])
data.OS = data.OS.apply(lambda x: reg.match(x).groups()[1])

In [157]:
data.head()

,Rating,MRP,Processor,RAM,OS,Storage,Brand,OS_arch
0,4.2,25490.0,Intel Celeron Dual Core Processor,4 GB LPDDR4 RAM,Windows 11,512 GB SSD,HP,64
1,3.8,22490.0,Intel Core i5 Processor,4 GB LPDDR4X RAM,Windows 11,512 GB SSD,HP,64
2,4.1,25685.0,Intel Celeron Dual Core Processor,8 GB DDR4 RAM,Windows 11,256 GB SSD,Lenovo,64
3,4.4,51990.0,Intel Core i5 Processor,8 GB DDR4 RAM,Windows 11,512 GB SSD,ASUS,64
4,4.2,38990.0,Intel Core i3 Processor,8 GB DDR4 RAM,Windows 11,512 GB SSD,ASUS,64


In [158]:
data.RAM.value_counts()

8 GB DDR4 RAM                                                                                             269
16 GB DDR4 RAM                                                                                            156
16 GB DDR5 RAM                                                                                             51
4 GB LPDDR4 RAM                                                                                            38
4 GB DDR4 RAM                                                                                              34
4 GB LPDDR4X RAM                                                                                           32
16 GB LPDDR5 RAM                                                                                           28
32 GB DDR5 RAM                                                                                             22
16 GB LPDDR4X RAM                                                                                          20
8 GB LPDDR

In [159]:
# drop the long one
data.RAM = data.RAM.replace("128 GB SSD for Reduced Boot Up Time and in Game LoadingUpgradable SSD Upto 512 GB and RAM8 GB DDR4 RAM", "8 GB DDR4 RAM")

In [160]:
reg = re.compile(r"(\d+)\s(GB)\s(.*)\s(RAM)")
reg.match("833 GB Unified Memory RAM").groups()

data["RAM_Type"] = data.RAM.apply(lambda x: reg.match(x).groups()[-2])
data.RAM = data.RAM.apply(lambda x: " ".join(reg.match(x).groups()[:2]))


In [161]:
data.head()

,Rating,MRP,Processor,RAM,OS,Storage,Brand,OS_arch,RAM_Type
0,4.2,25490.0,Intel Celeron Dual Core Processor,4 GB,Windows 11,512 GB SSD,HP,64,LPDDR4
1,3.8,22490.0,Intel Core i5 Processor,4 GB,Windows 11,512 GB SSD,HP,64,LPDDR4X
2,4.1,25685.0,Intel Celeron Dual Core Processor,8 GB,Windows 11,256 GB SSD,Lenovo,64,DDR4
3,4.4,51990.0,Intel Core i5 Processor,8 GB,Windows 11,512 GB SSD,ASUS,64,DDR4
4,4.2,38990.0,Intel Core i3 Processor,8 GB,Windows 11,512 GB SSD,ASUS,64,DDR4


In [162]:
data.Storage.value_counts()

512 GB SSD                      484
1 TB SSD                        120
256 GB SSD                       72
1 TB HDD256 GB SSD               24
2 TB SSD                         10
128 GB SSD                        3
128 GB SSD1 TB HDD128 GB SSD      2
1 TB HDD512 GB SSD                1
512 GB HDD512 GB SSD              1
4 TB SSD                          1
1 TB HDD128 GB SSD                1
1 TB HDD                          1
Name: Storage, dtype: int64

In [163]:
re.sub(r"\s([A-Z]+)(\d+)\s", r" \1,\2 ", "128 GB SSD1 TB HDD128 GB SSD")

'128 GB SSD,1 TB HDD,128 GB SSD'

In [164]:
reg = re.compile(r"\s([A-Z]+)(\d+)\s")
reg.sub(r" \1,\2 ", "128 GB SSD1 TB HDD128 GB SSD")
data.Storage = data.Storage.apply(lambda x: reg.sub(r" \1,\2 ", x))
data.Storage = data.Storage.apply(lambda x: x.split(','))
data.Storage.sample()

295    [512 GB SSD]
Name: Storage, dtype: object

In [165]:
data.to_csv("../data/preprocessed.csv")

In [166]:
data.head()

,Rating,MRP,Processor,RAM,OS,Storage,Brand,OS_arch,RAM_Type
0,4.2,25490.0,Intel Celeron Dual Core Processor,4 GB,Windows 11,[512 GB SSD],HP,64,LPDDR4
1,3.8,22490.0,Intel Core i5 Processor,4 GB,Windows 11,[512 GB SSD],HP,64,LPDDR4X
2,4.1,25685.0,Intel Celeron Dual Core Processor,8 GB,Windows 11,[256 GB SSD],Lenovo,64,DDR4
3,4.4,51990.0,Intel Core i5 Processor,8 GB,Windows 11,[512 GB SSD],ASUS,64,DDR4
4,4.2,38990.0,Intel Core i3 Processor,8 GB,Windows 11,[512 GB SSD],ASUS,64,DDR4


'512 GB SSD'